In [2]:
import os, subprocess, json
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

class DenseEncoder(object):
        
        def xlm_roberta_base_encoder(self, input_text):
                # Load the tokenizer and model
                tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
                model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
                tokens = tokenizer.encode(input_text, add_special_tokens=True, max_length=512, truncation=True)
                input_ids = torch.tensor([tokens])
                with torch.no_grad():
                        outputs = model(input_ids)
                        dense_vector = outputs[0][0][0]  # Extract the dense vector from the model output

                # Convert the dense vector to a numpy array
                dense_vector_list = dense_vector.numpy().tolist()
                return dense_vector_list

        
        def document_encoder(self):
                datasets_path = "/home/peng_luh/__github/search_l3s/search_l3s_search_srv/datasets"
                encodes_path = "/home/peng_luh/__github/search_l3s/search_l3s_search_srv/encodes"
                input_file_path = os.path.join(datasets_path, "mls-tasks-0/tasks.json")
                output_dir_path = os.path.join(encodes_path, "dense_encoder")
                
                with open(input_file_path) as f:
                        data = json.load(f)


                for d in data:
                        d["@id"] = d["id"]
                        d["id"] = int(d["id"].split("/")[-1])
                        d["vector"] = self.xlm_roberta_base_encoder(d["contents"])
                        d["contents"] = d["contents"] + "\n"

                        with open(f"{output_dir_path}/task.jsonl", "w") as jsonl_file:
                                for d in data:
                                        json.dump(d, jsonl_file)
                                        jsonl_file.write('\n')
                return


In [3]:
# from .logic import DenseEncoder

dense_enc = DenseEncoder()

dense_vec = dense_enc.xlm_roberta_base_encoder("Was is Elektrotechnik")

print(dense_vec)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[0.07217977941036224, 0.09157858043909073, 0.05999606475234032, -0.020205063745379448, 0.02170461416244507, -0.02944577857851982, 0.03564104810357094, 0.0004519230860751122, 0.06526193022727966, -0.11502823233604431, 0.011782891117036343, 0.06713750213384628, -0.06128199025988579, 0.006231870502233505, 0.004843269940465689, 0.051830656826496124, -0.03202732279896736, -0.01102643646299839, 0.08238693326711655, 0.058150727301836014, 0.028431113809347153, -0.005219425540417433, 0.08151587843894958, 0.04477493464946747, -0.013574089854955673, 0.03810984641313553, -0.05324321985244751, 0.05443398654460907, 0.06877885013818741, 0.013043617829680443, 0.09326209872961044, -0.015830939635634422, 0.04249339923262596, 0.07140932977199554, 0.07173920422792435, 0.017670203000307083, -0.112077496945858, -0.07562334090471268, 0.059521548449993134, 0.07727187126874924, 0.07298506796360016, 0.11304982006549835, 0.025053592398762703, -0.03082236647605896, 0.019926216453313828, -0.07809607684612274, -0.0

In [4]:
import os

dense_enc.document_encoder()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are ini

In [6]:
import subprocess

cmd2 = """
    python -m pyserini.index.faiss \
        --input /home/peng_luh/__github/search_l3s/search_l3s_search_srv/encodes/dense_encoder \
        --output /home/peng_luh/__github/search_l3s/search_l3s_search_srv/indexes/dense/hnsw \
        --hnsw
"""

subprocess.call(cmd2, shell=True)

100%|██████████| 1/1 [00:00<00:00,  9.17it/s]


(326, 768)
hnsw_add_vertices: adding 326 elements on top of 0 (preset_levels=0)
  max_level = 0
Adding 326 elements at level 0
Done in 21.725 ms
326


0

In [2]:
import os

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

folder_path = "/home/peng_luh/__github/search_l3s/search_l3s_search_srv/indexes/bm25/mls-tasks-1"

# Get the size of the folder in bytes
folder_size = get_folder_size(folder_path)

# Convert to appropriate units (KB, MB, GB, etc.)
folder_size_kb = folder_size / 1024
folder_size_mb = folder_size_kb / 1024
folder_size_gb = folder_size_mb / 1024

# Print the folder size
print(f"Folder size: {folder_size} bytes ({folder_size_kb:.2f} KB, {folder_size_mb:.2f} MB, {folder_size_gb:.2f} GB)")

Folder size: 9592 bytes (9.37 KB, 0.01 MB, 0.00 GB)


In [1]:
import os

if os.path.exists("/home/peng_luh/__github/search_l3s/search_l3s_search_srv/encodes/dense/xlm_roberta_base/mls-tasks-0/data_encoded.jsonl"):
    print("True")
else:
    print("False")

True


In [2]:
import json

with open("/home/peng_luh/__git/search_l3s/search_l3s_search_srv/datasets/mls-tasks-test/json/tasks2.json") as f:
    data = json.load(f)

print(data)

[{'id': '/mls-api/tasks/418', 'contents': 'Kick-off Grundlagen der Sensorik - Schwingungssensor; Cover; Sicherheitshinweise; Einführung; Projektvorstellung\n'}, {'id': '/mls-api/tasks/417', 'contents': 'Grundlagen der Sensorik: Schwingungssensor - Betrieblicher Auftrag; Erläuterung; Aufgabenstellung; Vereinfachungen/Simulationen; Aufbau und Messanordnung; Nachweis der Funktion\n'}, {'id': '/mls-api/tasks/416', 'contents': 'Grundlagen der Sensorik: Schwingungssensor - Arbeitsplanung; Arbeitsvorbereitung; Eingesetzter Sensor; Datenblatt; Tastenprogrammierung und Grenzwerte festlegen\n'}, {'id': '/mls-api/tasks/415', 'contents': 'Halbleiterbauelemente der Elektronik Versuch 2.2: Operationsverstärker als Impedanzwandler und summierender Verstärker; Impedanzwandler; Impedanzwandler (Spannungsfolger); Summierender Verstärker; Summierer in der praktischen Übung; Summierer mit Gleich- und Wechselspannung\n'}, {'id': '/mls-api/tasks/414', 'contents': 'Halbleiterbauelemente der Elektronik Versuc

In [7]:
import subprocess
import transformers

cmd = """
    python -m pyserini.encode \
        input   --corpus /home/peng_luh/__git/search_l3s/search_l3s_search_srv/datasets/mls-tasks-test/jsonl/data.jsonl \
                --fields text \
                --delimiter ";" \
                --shard-id 0 \
                --shard-num 1 \
        output  --embeddings /home/peng_luh/__git/search_l3s/search_l3s_search_srv/encodes/dense/xlm_roberta_base/mls-tasks-test \
                --to-faiss \
        encoder --encoder xlm-roberta-large \
                --fields text \
                --batch 32 \
                --fp16
"""

subprocess.call(cmd, shell=True)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Traceback (most recent call last):
  File "/home/peng_luh/.pyenv/versions/3.9.5/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/peng_luh/.pyenv/versions/3.9.5/lib/python3.9/runpy.py", line 87, in _run_code
    ex

1

In [1]:
from search_l3s_search.api.encoder.logic import BertGermanUncasedDenseEncoder

enc = BertGermanUncasedDenseEncoder()

enc.print_model_name()

enc.dataset_encoder(dataset_name="mls-tasks")

/home/peng_luh/__git/search_l3s/search_l3s_search_srv/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initial

bert_german_uncased
/home/peng_luh/__git/search_l3s/search_l3s_search_srv/datasets/mls-tasks/jsonl/data.jsonl
/home/peng_luh/__git/search_l3s/search_l3s_search_srv/encodes/dense/bert_german_uncased/mls-tasks


1

In [7]:
from search_l3s_search.api.encoder.logic import BertGermanUncasedDenseEncoder

enc = BertGermanUncasedDenseEncoder()

print(len(enc.query_encoder("Beispiel")))

enc.dataset_encoder(dataset_name="mls-tasks")

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation 

768
/home/peng_luh/__git/search_l3s/search_l3s_search_srv/datasets/mls-tasks/jsonl/data.jsonl
/home/peng_luh/__git/search_l3s/search_l3s_search_srv/encodes/dense/bert_german_uncased/mls-tasks


1

In [11]:
import os, subprocess
def hnsw_indexer(encode_cat, model_name, dataset_name):
        dataset_encode_path = os.path.join(os.getcwd(),
                                            f"encodes/{encode_cat}/{model_name}/{dataset_name}"
                                        )
        if not os.path.exists(dataset_encode_path):
            raise FileNotFoundError
        
        output_path = os.path.join(os.getcwd(), f"indexes/{encode_cat}/{model_name}/hnsw/{dataset_name}")
        # print(output_path)
        
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        hnsw_cmd = f"""
            python -m pyserini.index.faiss \
                --input {dataset_encode_path} \
                --output {output_path} \
                --hnsw
        """
        
        subprocess.call(hnsw_cmd, shell=True)
        return 1

In [12]:
hnsw_indexer(encode_cat="dense", model_name="bert_german_uncased", dataset_name="mls-tasks")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00,  7.92it/s]


(326, 768)
hnsw_add_vertices: adding 326 elements on top of 0 (preset_levels=0)
  max_level = 0
Adding 326 elements at level 0
Done in 40.108 ms
326


1

In [19]:
from pyserini.search.faiss import FaissSearcher
import json

def dense_retrieval(query):
        model_name = "bert_german_uncased"
        dataset_name = "mls-tasks"
        index_method = "hnsw"
        
        dataset_file_path = os.path.join(os.getcwd(), f"datasets/{dataset_name}/jsonl/data.jsonl")
        
        prebuilt_index_path = os.path.join(os.getcwd(), f"indexes/dense/{model_name}/{index_method}/{dataset_name}")

        search_engine = FaissSearcher(
            prebuilt_index_path,
            "dbmdz/bert-base-german-uncased"
        )
                
        hits = search_engine.search(query)
        results=[]
        
        # if hits:
        #     for i in range(0, len(hits)):
        #         temp = ast.literal_eval(hits[i].raw)
        #         temp['score'] = f'{hits[i].score:.4f}'
        #         results.append(temp)
        
        for i in range(0, 10):
            docid = hits[i].docid
            with open(dataset_file_path, "r") as dataset:
                for line in dataset:
                    json_obj = json.loads(line)
                    if int(docid) == json_obj["id"]:
                        temp = dict()
                        temp["@id"] = json_obj["@id"]
                        temp["contents"] = json_obj["contents"]
                        temp["score"] = f"{hits[i].score:.5f}"
                        results.append(temp)
            
            # print(f'{i+1:2} {hits[i].docid:7} {hits[i].score:.5f}')
            
        return results

In [17]:
embedding_331 = [0.22154638171195984, -0.03536605462431908, -0.2912493050098419, 0.5319214463233948, -0.36612260341644287, 0.03568511828780174, 0.27370965480804443, 0.00978295411914587, -0.045464105904102325, -0.267067551612854, 0.6439609527587891, -0.1166854053735733, -0.7110666632652283, 0.9173524379730225, 0.4479750990867615, 0.22522468864917755, -0.5504530668258667, -0.3796117305755615, -0.41885364055633545, -0.5035166144371033, 0.18854443728923798, 0.23881149291992188, 1.1101657152175903, -0.9617190361022949, -0.006559269968420267, 0.2220858633518219, -0.22361570596694946, -0.18334241211414337, -0.03386091813445091, 0.6254233717918396, 0.40355944633483887, -0.6828979253768921, 0.5137401819229126, 0.5067671537399292, -0.3722723424434662, 0.2831169664859772, -0.24351198971271515, 0.3541296720504761, 0.5935749411582947, -0.11793436110019684, 0.2677404582500458, -0.2652527391910553, 0.5108146071434021, -0.5921666622161865, -0.17076407372951508, -0.4888835847377777, -0.048121627420186996, -1.0304791927337646, -0.02815260738134384, -0.24148282408714294, -0.6323826313018799, 0.08970674127340317, 0.46729499101638794, -0.27126574516296387, -0.16023801267147064, 0.2958875894546509, 0.2974606454372406, 0.6791817545890808, -0.027798665687441826, 0.8141459822654724, 0.7901440858840942, -0.19421859085559845, -0.06948456913232803, -0.021152153611183167, 0.36398953199386597, -0.4986671507358551, -0.3841012120246887, -1.1012091636657715, -0.18721577525138855, 0.05288136005401611, -0.16339851915836334, 0.2620474696159363, 0.4209460914134979, -3.2898201942443848, 0.13884419202804565, 0.49751394987106323, 1.4624335765838623, 0.4739340543746948, 0.32848060131073, -0.6530799865722656, 0.07329990714788437, 0.031008083373308182, -0.8977426886558533, 0.06638327240943909, 0.011134698055684566, 0.6172598600387573, -1.379198670387268, 0.2541881799697876, 0.2216394692659378, -0.4148119390010834, 0.2917603552341461, -0.3300979435443878, -0.03924437239766121, -0.9383015036582947, -0.020288363099098206, -0.07907771319150925, 0.09857144951820374, -0.3385886549949646, -0.4194817841053009, 0.32148635387420654, -0.07520481944084167, -0.2854905426502228, -0.20120887458324432, -0.3893740475177765, 0.23189781606197357, 0.20734094083309174, 0.22193670272827148, 0.46635064482688904, 0.9917510747909546, -0.3811907470226288, -0.08535239845514297, -0.13135433197021484, 0.3366360664367676, 0.8447031378746033, 0.5194770097732544, 0.4730754792690277, -0.10181786119937897, 0.4108662009239197, 0.563346803188324, -0.004596702288836241, 0.10382963716983795, 0.1976446956396103, -0.4171176552772522, -0.01944463886320591, -0.6670040488243103, -0.20813995599746704, 0.25109803676605225, 0.29040631651878357, 0.19775390625, -0.806316077709198, -1.5847177505493164, 0.287110298871994, 0.6104015111923218, 0.06437893956899643, -0.37466368079185486, 0.3771003782749176, 0.137516051530838, 0.3757713735103607, 1.0917292833328247, -0.17437972128391266, 0.26122725009918213, -0.004995782393962145, -0.5880810618400574, 0.6752739548683167, 0.1093020811676979, 0.2423037886619568, -0.3378508687019348, -0.10573738068342209, -0.3565458357334137, -14.043425559997559, 0.47333618998527527, 0.9879235625267029, -0.5170160531997681, -1.191133975982666, 0.4889661371707916, 0.25763139128685, -0.03934541717171669, -0.4841372072696686, -0.06555373221635818, 0.05024592950940132, -0.8585819602012634, -0.1499122679233551, -0.47456884384155273, -0.8169113993644714, 0.4768712818622589, -0.09233439713716507, -0.06613187491893768, 0.37947332859039307, -0.5797991156578064, 0.2505777180194855, 0.47341272234916687, 0.35033443570137024, -1.1364407539367676, 0.2004738301038742, -0.08153394609689713, -0.34715622663497925, -0.33537548780441284, 0.47172775864601135, 0.05629431828856468, -0.7562210559844971, 0.2860223650932312, 0.33384695649147034, 0.7083267569541931, -0.13190196454524994, 1.2097312211990356, -0.28878292441368103, -0.10386764258146286, -0.5908347964286804, 0.47809305787086487, 0.2522825598716736, 0.03384004160761833, 0.1476939618587494, 0.16521002352237701, -0.037338756024837494, 0.4095965027809143, -0.3498218357563019, 0.03930428624153137, -0.04635385423898697, 0.9078057408332825, 0.2618737518787384, 0.5488669872283936, 0.015335286036133766, -0.29952970147132874, -0.506941556930542, -0.010026415809988976, -0.06746960431337357, 0.15556350350379944, 0.2657564580440521, -1.2192751169204712, 0.18280719220638275, -0.06940429657697678, -0.2720226049423218, -0.5463625192642212, 0.43049514293670654, -0.3362327814102173, 0.08873749524354935, 0.24111127853393555, 0.13457022607326508, 1.2211908102035522, 0.4936220943927765, 0.2184792459011078, 0.36253029108047485, 0.45400264859199524, 0.2977726459503174, -0.10401637107133865, 0.18768416345119476, 0.21402980387210846, 0.22453179955482483, 0.03527450934052467, -0.09471135586500168, -0.191964790225029, -0.40598565340042114, -0.009422004222869873, 0.235077366232872, -0.913417398929596, 0.2609175741672516, 0.4095500111579895, 0.2356719821691513, -0.1571531444787979, 0.8467679023742676, 0.038500506430864334, -0.271562397480011, 0.318322092294693, -0.3009224534034729, 0.0237403754144907, 0.45458608865737915, -0.06051953136920929, -0.25164952874183655, 0.33925238251686096, 0.2361641526222229, -0.39735147356987, 0.08137153089046478, -0.19485220313072205, -0.3810477554798126, 0.009734799154102802, -0.25898462533950806, 0.7409868836402893, -0.07437841594219208, 0.0029077017679810524, 0.47381719946861267, 0.2935835123062134, 0.11488308012485504, 0.2173895388841629, -0.5445263981819153, -0.261358380317688, -0.09375723451375961, -0.2329961657524109, -0.3792358636856079, 0.8232138752937317, -0.15006932616233826, 0.020131032913923264, 0.01216090191155672, -0.6171147227287292, 0.1832321286201477, 0.5187524557113647, 0.011182336136698723, -0.009324604645371437, 0.27990487217903137, 0.20388612151145935, 0.5358164310455322, 0.07206989079713821, 0.9242681264877319, -0.263177752494812, 0.6874938607215881, 0.3132344186306, 0.1495007425546646, 0.01707719825208187, 0.3223930895328522, -0.650538980960846, -0.14148642122745514, -0.4693257212638855, 0.09812896698713303, 0.0019449596293270588, 0.05856376886367798, -0.8004976511001587, -0.468003511428833, 0.42769551277160645, -0.3013893663883209, -0.0921923890709877, 0.14745904505252838, 0.6136793494224548, 0.6398677825927734, -0.12428512424230576, -0.12186910957098007, -0.6339580416679382, -0.3424898386001587, -0.2910423278808594, 0.32909417152404785, -0.08585228025913239, 0.13842763006687164, 0.411792516708374, 0.534913957118988, 0.42631080746650696, -0.22637243568897247, -0.08645687997341156, 0.13470439612865448, -0.8058285713195801, 0.2505238950252533, -0.0877099558711052, 0.2665371298789978, -0.3572221100330353, 0.37013161182403564, -0.7641989588737488, 0.8427732586860657, -0.36719581484794617, 0.017611632123589516, -0.28877130150794983, 1.0331522226333618, -0.7090998888015747, -0.08541853725910187, -0.14591820538043976, -0.16174393892288208, 0.8785203099250793, -0.6647529006004333, -1.1045868396759033, 0.010873427614569664, 0.01748516783118248, -0.38860464096069336, -0.4298636019229889, -0.500834584236145, 0.7375121116638184, -0.07536491006612778, 0.6416773796081543, -0.386107474565506, -0.19342394173145294, -0.46380215883255005, -0.04931049421429634, 0.43778252601623535, 0.32846617698669434, -0.04507628455758095, 0.9580941200256348, 1.0597399473190308, 0.0874054878950119, -0.41313958168029785, 0.5303297638893127, 0.11317025125026703, -0.3927030563354492, -0.5442252159118652, 0.7966984510421753, -0.1298971176147461, -0.20649994909763336, -0.946942150592804, 0.021568890661001205, 1.1890907287597656, -1.2584587335586548, -0.33032625913619995, -0.5287941098213196, 0.1044955775141716, 0.6208651065826416, 0.3320682644844055, 0.3645220100879669, 0.31699275970458984, -0.254189670085907, -0.5772091150283813, 0.09218815714120865, -0.20067502558231354, -0.1096423864364624, -0.4546753466129303, -0.22713476419448853, -0.07566504925489426, -0.12437400221824646, 0.597306489944458, -0.27569589018821716, -0.33553454279899597, 0.6672300100326538, -0.37848639488220215, 0.1949804127216339, -0.18572303652763367, -0.10126736015081406, -0.1346205174922943, -0.13155597448349, -1.0336236953735352, 0.10716253519058228, -0.5330153107643127, 0.7319086194038391, 0.5263681411743164, -0.23031586408615112, -0.5921040773391724, 0.0830376073718071, -0.050373226404190063, -0.21228475868701935, -0.7342960834503174, -0.39397627115249634, 0.06279758363962173, -0.04762892797589302, -0.17493334412574768, -0.5439286828041077, 0.3184834122657776, -0.5473164916038513, 0.15268610417842865, 0.18267792463302612, 0.21561913192272186, 0.5564021468162537, 0.05969442427158356, -0.3835783302783966, -1.075946569442749, -0.23557469248771667, 0.19136376678943634, -0.289767324924469, -0.08182723820209503, -0.3794582486152649, 0.06350813806056976, 0.6975003480911255, -0.15292294323444366, 0.343221515417099, 0.22280757129192352, -0.513473629951477, -0.30218952894210815, -0.42250365018844604, 0.1660229116678238, -0.30724847316741943, 0.21209849417209625, -0.8669721484184265, 0.9081754684448242, 0.45162251591682434, 1.4697083234786987, 0.3743985593318939, 0.1878405511379242, -0.16802945733070374, 0.44648921489715576, 1.2136332988739014, 0.09858949482440948, 0.31655964255332947, 0.07666774094104767, 0.19530776143074036, 0.10853416472673416, -0.09794075787067413, 0.1263394057750702, 0.004142037592828274, -0.6776214838027954, -0.3137854039669037, 0.1892148107290268, 0.06586449593305588, 0.03050629049539566, 0.2779353857040405, -0.26581695675849915, -0.12435257434844971, -0.11469384282827377, 0.19596610963344574, 0.4760918617248535, 0.16739672422409058, 0.36044934391975403, 0.20693767070770264, 0.600701630115509, 0.05300360918045044, -0.6535438299179077, 0.29670313000679016, -0.2649935483932495, 0.3100591003894806, -0.16534222662448883, 0.603947103023529, -0.1965954452753067, 0.22026464343070984, 0.5556516051292419, 0.3796635568141937, -1.0347172021865845, -0.6868613362312317, -0.5368090867996216, -0.1719871312379837, 0.12198743224143982, -0.5445279479026794, -0.16824106872081757, 0.0002375345939071849, -0.18796195089817047, -0.45415955781936646, -0.42497947812080383, -0.4615939259529114, -0.20420196652412415, 0.3073006570339203, 0.030007638037204742, -0.7294412851333618, -0.3453335165977478, -0.5656929016113281, -0.5996313095092773, -0.30348968505859375, -0.20380379259586334, 0.007559905294328928, -0.45595479011535645, -0.07570259273052216, 0.03928603231906891, -0.1769038885831833, -0.13529008626937866, 1.1439050436019897, -0.33488404750823975, -0.41670677065849304, 0.3602760434150696, -0.0847148448228836, -0.25955817103385925, -0.2038497030735016, -0.2561943531036377, 1.0978432893753052, 0.20240794122219086, 0.461073637008667, 0.1298782080411911, -0.3727109432220459, 0.7304834127426147, 0.5297349691390991, -0.551512598991394, 0.7137865424156189, 0.10095083713531494, -0.013575551100075245, -0.20321029424667358, -0.4811446964740753, -0.2997974455356598, -0.37714141607284546, -1.0301226377487183, 0.022593699395656586, 0.6810116171836853, -0.05781028792262077, 0.24921448528766632, 0.5631072521209717, 0.5052714943885803, -0.5453206896781921, 0.1162337139248848, 0.2852400839328766, -0.1939760446548462, 0.4778609573841095, 0.17707528173923492, 0.20232945680618286, 0.6441445350646973, -0.32558736205101013, -0.3608724772930145, -0.3715103566646576, 0.11373715102672577, 0.04518590122461319, 0.7240031361579895, -0.08473435789346695, 0.9469995498657227, 0.2008855640888214, -0.01693122088909149, 0.30899757146835327, 0.5058563947677612, 0.5853034853935242, -0.7788946032524109, -0.32663822174072266, 0.720164954662323, -0.05127892270684242, 0.09994880855083466, 0.20955879986286163, 0.28164759278297424, 1.2397141456604004, 0.22406895458698273, -0.16704663634300232, 0.23139506578445435, 0.16772791743278503, 0.12261948734521866, 0.4314460754394531, -0.45407983660697937, -0.44550782442092896, 0.15390868484973907, 0.0610174685716629, 0.41923561692237854, -0.49442362785339355, 0.48522162437438965, -0.7421517372131348, 0.03433595970273018, 0.0006315102800726891, -0.41982829570770264, -0.1552351415157318, 0.13251811265945435, -0.6308271288871765, -1.5918852090835571, -0.48867014050483704, 0.8102247714996338, -0.68392413854599, 0.13340236246585846, -0.40186306834220886, 0.7239654064178467, 0.45854106545448303, -0.34783491492271423, 0.6744083166122437, 0.004725173115730286, -0.13768263161182404, 0.11462234705686569, 0.6968070864677429, 0.754035234451294, 0.2863507568836212, 1.0739479064941406, -0.04527704790234566, 0.42607423663139343, -0.16836144030094147, 0.8456312417984009, 0.2766447067260742, -0.5625872611999512, 0.539954423904419, 0.11684135347604752, 0.5189899802207947, -0.43748271465301514, 0.20900914072990417, 1.0194834470748901, 0.22133643925189972, 0.2640106678009033, -0.41443586349487305, 0.5090633630752563, -0.37466493248939514, -0.06732446700334549, 0.8816194534301758, 0.19706715643405914, 0.33617717027664185, -0.3937286138534546, 0.26783373951911926, -0.6894789934158325, 0.843172013759613, 0.2604798674583435, 0.6511631608009338, 0.5821529626846313, 0.3069513142108917, 0.17376618087291718, 0.18571771681308746, 0.32309484481811523, 0.47388768196105957, -0.3340679407119751, -0.10960562527179718, 0.34419724345207214, -0.1232864260673523, -0.443847119808197, 0.7879575490951538, 0.3149968683719635, 0.1349290907382965, -0.5409595966339111, 0.151164710521698, -0.14085401594638824, 0.276419460773468, 0.3630043566226959, -0.449211061000824, -0.6947718262672424, 0.5284052491188049, -0.39295142889022827, 0.4084552824497223, 0.5665998458862305, 0.7901442050933838, -0.25017213821411133, 0.22328875958919525, -0.11152839660644531, 0.6064267158508301, 0.3957063853740692, -0.03258167579770088, -0.03688677400350571, -0.7916764616966248, 0.18453571200370789, -0.10405879467725754, -0.2748679518699646, -0.02709329128265381, -0.6729950308799744, 1.5267003774642944, 0.5404310822486877, 0.6259427070617676, 0.07243498414754868, 0.03122643753886223, 0.832466185092926, 0.05452905222773552, -0.28396075963974, 0.22088930010795593, -0.16975024342536926, -0.35301029682159424, -0.07598623633384705, 0.07032258808612823, -0.8090739846229553, -0.14253629744052887, 0.76167231798172, -0.04696652293205261, -1.210944652557373, 0.4921262860298157, 0.16249603033065796, -0.12937864661216736, 0.8671388030052185, -0.40338319540023804, 0.3843778073787689, -0.7210575938224792, -0.3896016776561737, -0.012906895019114017, -0.26147031784057617, 0.10315210372209549, -0.6329640746116638, -0.5726400017738342, 1.100956678390503, -0.3385281562805176, 0.1337314248085022, -0.4345676898956299, 0.6876882910728455, 0.04344357177615166, -0.17304807901382446, 0.25321730971336365, -0.8290703296661377, 1.0179016590118408, 0.26629671454429626, 0.31281203031539917, -0.030012033879756927, 0.654255747795105, -1.029093861579895, 0.5767906904220581, -0.47162967920303345, -0.026003235951066017, 0.29319819808006287, -0.6430271863937378, -0.09829962253570557, -0.6163454055786133, 0.2679811716079712, 0.12214057892560959, 0.4514056146144867, 0.244789257645607, 0.33376190066337585, -0.13890382647514343, 0.15278877317905426, 0.29844173789024353, -0.147852823138237, -0.3871980309486389, -0.5550984144210815, -0.38286474347114563, -0.255177766084671, -0.1669131964445114, -0.59510737657547, -0.09419433772563934, -0.09870947152376175, 0.5180899500846863, 0.027720343321561813, 0.06511766463518143, -0.1658174693584442, 0.4609411656856537, -1.0799025297164917, -0.5820200443267822, -0.12321826815605164, -1.3496798276901245, -0.5268656611442566, 0.3543970286846161, -0.10355700552463531, 0.13736726343631744, -0.2346586436033249, -0.01607373170554638, 1.2557207345962524, -0.18556655943393707, 0.19090339541435242, 0.6681014895439148, 0.2220887839794159, -0.16175973415374756, -0.0326644666492939, -0.5007616877555847, -0.38724035024642944, 0.3141181170940399, -0.1938156634569168, 0.17886663973331451, 0.19278892874717712, -0.38518935441970825, -0.7564964294433594, 0.39979851245880127, 0.2148739993572235, 0.533394992351532, -1.0600650310516357]

In [18]:
from pprint import pprint
# 331
query = "Kick-off Regelungstechnik - Grundlagen; Cover; Sicherheitshinweise; Einf\u00fchrung und Ablauf; Versuchsaufbau"
embdding_query = enc.query_encoder(query)
print(embdding_query)

if embedding_331 == embdding_query:
    print(True)
else:
    print(False)
    
    
results = dense_retrieval(query)

pprint(results)

[102, 17756, 232, 15008, 7591, 11634, 232, 12298, 3464, 18666, 3464, 4367, 1176, 875, 3464, 5091, 136, 11137, 3464, 21074, 21094, 103]
[0.22154638171195984, -0.03536605462431908, -0.2912493050098419, 0.5319214463233948, -0.36612260341644287, 0.03568511828780174, 0.27370965480804443, 0.00978295411914587, -0.045464105904102325, -0.267067551612854, 0.6439609527587891, -0.1166854053735733, -0.7110666632652283, 0.9173524379730225, 0.4479750990867615, 0.22522468864917755, -0.5504530668258667, -0.3796117305755615, -0.41885364055633545, -0.5035166144371033, 0.18854443728923798, 0.23881149291992188, 1.1101657152175903, -0.9617190361022949, -0.006559269968420267, 0.2220858633518219, -0.22361570596694946, -0.18334241211414337, -0.03386091813445091, 0.6254233717918396, 0.40355944633483887, -0.6828979253768921, 0.5137401819229126, 0.5067671537399292, -0.3722723424434662, 0.2831169664859772, -0.24351198971271515, 0.3541296720504761, 0.5935749411582947, -0.11793436110019684, 0.2677404582500458, -0.26

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AttributeError: 'function' object has no attribute 'lower'

In [8]:
# fetch the embedding as list
import json, numpy

sentence_embedding = []

with open("./encodes/dense/bert_german_uncased/mls-tasks/data_encoded.jsonl") as data:
    for line in data:
        json_obj = json.loads(line)
        sentence_embedding.append(json_obj["vector"])

print(type(sentence_embedding))

print(numpy.array(sentence_embedding).shape)




<class 'list'>
(326, 768)


In [30]:
import faiss
sentence_embedding = numpy.array(sentence_embedding)
index = faiss.IndexFlatL2(768)

In [31]:
index.add(numpy.float32(sentence_embedding))

In [32]:

k = 10

xq = numpy.float32(numpy.array([enc.query_encoder("Montage- und Demontageautomat Versuch 5: Vollfunktion Montieren/Demontieren; Beschreibung; Erweiterungen des Aufbaus; Ablaufbeschreibung; Ablaufgraph; Programmerstellung in FUP")]))

# Distance, Labels
D, I = index.search(xq, k)  # search
print(I)
print(D)

[[ 71  72  73  74 185  78 143 123 266 137]]
[[ 0.        7.625406  8.257185 10.935714 46.676605 49.768665 55.11333
  55.84443  55.84443  56.041443]]


In [29]:
embd = sentence_embedding[71].tolist()

with open("./encodes/dense/bert_german_uncased/mls-tasks/data_encoded.jsonl") as data:
    for line in data:
        json_obj = json.loads(line)
        if json_obj["vector"] == embd:
            print(json_obj["@id"])

/mls-api/tasks/347


In [38]:
import json

f = open("/home/peng_luh/__git/search_l3s/search_l3s_search_srv/datasets/mls-tasks/json/data.json")

data = json.load(f)

print(type(data))

lst = [62, 114, 206, 105, 118, 218, 192, 270, 230, 133]

print(data[62])

f.close

<class 'list'>
{'id': '/mls-api/tasks/356', 'contents': 'Ihr Feedback an uns; Willkommen; Feedbackbogen'}


<function TextIOWrapper.close()>

In [39]:
o = open("/home/peng_luh/__git/search_l3s/search_l3s_search_srv/indexes/dense/bert_german_uncased/flat/mls-tasks/docid")

docid = o.read()

print(docid)

[418, 417, 416, 415, 414, 413, 412, 411, 410, 409, 408, 407, 406, 405, 404, 403, 402, 401, 400, 399, 398, 397, 396, 395, 394, 393, 392, 391, 390, 389, 388, 387, 386, 385, 384, 383, 382, 381, 380, 379, 378, 377, 376, 375, 374, 373, 372, 371, 370, 369, 368, 367, 366, 365, 364, 363, 362, 361, 360, 359, 358, 357, 356, 355, 354, 353, 352, 351, 350, 349, 348, 347, 346, 345, 344, 343, 342, 341, 340, 339, 338, 337, 336, 335, 334, 333, 332, 331, 330, 329, 328, 327, 326, 325, 324, 323, 322, 321, 320, 319, 318, 317, 316, 315, 314, 313, 312, 311, 310, 309, 308, 307, 306, 305, 304, 303, 302, 301, 300, 299, 298, 297, 296, 295, 294, 293, 292, 291, 290, 289, 288, 287, 286, 285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273, 272, 271, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253, 252, 251, 250, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219,